# Домашнее задание #2

## Общая информация

**Дата выдачи:** 2022.11.11

**Дедлайн:** 2022.11.30 (ср), 23:55

# 0. Подготовка к путешествию

In [1]:
import pandas as pd

from google.colab import data_table
data_table.enable_dataframe_formatter()

from tqdm.notebook import tqdm
tqdm.pandas()

import time
import random

import numpy as np

!nvidia-smi

Sat Nov 19 23:02:55 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
df = pd.read_csv('/content/drive/MyDrive/Notebooks/NLP/HW2_NMT/df.csv', encoding = 'utf8')

In [ ]:
! cd /content/drive

# 1. **[3 балла]** Использование готовых моделей перевода и оценка качества

В этом задании вам необходимо перевести тексты с русского языка на английский при помощи готовой модели машинного перевода а также известного онлайн-переводчика. Требуется сравнить результаты работы обоих методов используя метрики [BLEU](https://huggingface.co/spaces/evaluate-metric/bleu) и [BLEURT](https://huggingface.co/spaces/evaluate-metric/bleurt). В качестве данных будем использовать датасет [ted_talks_iwslt](https://huggingface.co/datasets/ted_talks_iwslt).

Скачаем данные и посмотрим на них:

In [ ]:
! pip install pandas tqdm datasets

from datasets import load_dataset

raw_data = load_dataset("ted_talks_iwslt", language_pair=("ru", "en"), year="2016")
df = pd.DataFrame(raw_data['train']['translation'])
print(f"Dataset contains {len(df)} samples")

In [10]:
from pathlib import Path  

filepath = Path('/content/drive/MyDrive/Notebooks/NLP/HW2 NMT/df.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True)  
df.to_csv(filepath) 

### **[0.5 балла]** Датасет содержит 4100 пар переводов текстов с русского языка на английский. В этом задании вам необходимо перевести предложения с русского на английский при помощи одной из готовых моделей из библиотеки [EasyNMT](https://github.com/UKPLab/EasyNMT).

In [ ]:
df.head(1)

,en,ru
0,If you've taken a career break and are now loo...,"Если вы взяли перерыв в карьере, но уже хотите..."


In [6]:
!pip install easynmt

from easynmt import EasyNMT

model = EasyNMT('opus-mt')

11.9kB [00:00, 18.7MB/s]                   


In [ ]:
# sentences = df['ru'].tolist()
# df['opus-mt'] = model.translate(sentences, source_lang='ru', target_lang='en',
#                                  show_progress_bar=True)

df['opus-mt'] = df['ru'].progress_apply(lambda x: model.translate(x, source_lang='ru',
                                                                  target_lang='en',
                                                                  beam_size=2))

# beam_size=2 for faster inference

In [11]:
df.to_csv(filepath)

In [ ]:
df.head(1)

,en,ru,opus-mt
0,If you've taken a career break and are now loo...,"Если вы взяли перерыв в карьере, но уже хотите...","If you've taken a break in your career, but yo..."


### **[0.5 балла]** Переведите тот же самый набор предложений с помощью Google Translate. Для этого воспользуйтесь библиотекой [googletrans](https://pypi.org/project/googletrans/).

In [ ]:
#@title user-agent

# !pip3 install fake-useragent

from fake_useragent import UserAgent
ua = UserAgent()

In [ ]:
# !pip3 install googletrans==4.0.0rc1

from googletrans import Translator

translator = Translator(user_agent='ua')

translator.raise_Exception = True

df['googletrans'] = df['ru'].progress_apply(lambda x: translator.translate(x, dest='en').text)

# sentences = df['ru'].to_list()
# translations = []
# cnt = 0

# for sen in tqdm(sentences):
#     cnt += 1
#     translations.append(translator.translate(sen, dest='en').text)
#     if cnt % 10 == 0:
#       time.sleep(random.randint(0.5, 1.5))

* google забанил меня, решил через время попробовать другую библиотеку -- **deep_translator**

In [ ]:
from deep_translator import GoogleTranslator

translator = GoogleTranslator(source='ru', target='en')

df['googletrans'] = df['ru'].progress_apply(lambda x: translator.translate(x))

  0%|          | 0/4100 [00:00<?, ?it/s]

In [ ]:
df.to_csv(filepath)

###  **[0.5 балла]** Сравните полученные результаты с ground truth переводом. Какие ошибки вы можете наблюдать, с чем они могут быть связаны? Какой перевод выглядит более качественным по вашему мнению?

In [ ]:
df.sample(3)

,Unnamed: 0,en,ru,opus-mt,googletrans
1590,1590,"When a new drug gets tested, the results of th...",Результаты исследований новых лекарств должны ...,The results of research on new drugs should be...,The results of studies of new drugs should be ...
2732,2732,"In graphic design, Marian Bantjes says, throwi...","Мариан Бантйес говорит, что в графическом диза...",Marian Buntyes says that in graphic design no ...,"Marian Bantjes says that in graphic design, no..."
2613,2613,Mechai Viravaidya: How Mr. Condom made Thailan...,Мечаи Вираваидия: Как Мистер Презерватив сдела...,How did Mr. Condomative make Thailand a better...,Mechai Viravaidiya: How Mr. Condom made Thaila...


Считаю, что ground truth перевод ненамного опережает машинный перевод, но все же есть пару отличительных моментов:
*   Модели NMT плохо справляются с переводом устойчивых выражений, такой перевод получается более "прямолинейным".
*   Моделями NMT допущено много ошибок в написании имен, фамилий. 





### **[0.75 балла]** Сравните качество перевода используя метрики BLEU и BLEURT. Для этого воспользуйтесь библиотекой [evaluate](https://github.com/huggingface/evaluate). Опишите принцип подсчета этих метрик, чем они отличаются, какие имеются преимущества и недостатки у каждой из этих метрик?

In [ ]:
!pip install evaluate

  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [ ]:
! pip install git+https://github.com/google-research/bleurt.git

In [ ]:
df.head(1)

,Unnamed: 0,en,ru,opus-mt,googletrans
0,0,If you've taken a career break and are now loo...,"Если вы взяли перерыв в карьере, но уже хотите...","If you've taken a break in your career, but yo...",If you've taken a break from your career but a...


In [ ]:
import evaluate
from bleurt import score
# mt_metrics = evaluate.combine(["bleu", "bleurt"])

In [ ]:
bleu = evaluate.load("bleu")
bleurt = evaluate.load("bleurt")

In [ ]:
x = bleu.compute(predictions=df['opus-mt'].to_list(),
                   references=df['en'].to_list())


y = bleurt.compute(predictions=df['opus-mt'].to_list(),
                   references=df['en'].to_list())

# results = mt_metrics.compute(predictions=df['opus-mt'].to_list(),
#                              references=df['en'].to_list(),
#                              sources=df['ru'].to_list())

In [ ]:
np.array(y['scores']).mean()

0.04607125436387411

### **[0.75 балла]** Модели из EasyNMT поддерживают параметр beam_size. За что отвечает этот параметр? Попробуйте получить переводы текстов с различным значением параметра beam_size? Как меняется качество в зависимости от значения параметра? Опишите полученные результаты.



In [8]:
df['opus-mt_bz=5'] = df['ru'].progress_apply(lambda x: model.translate(x, source_lang='ru',
                                                                  target_lang='en',
                                                                  beam_size=5))

df['opus-mt_bz=10'] = df['ru'].progress_apply(lambda x: model.translate(x, source_lang='ru',
                                                                  target_lang='en',
                                                                  beam_size=10))

  0%|          | 0/4100 [00:00<?, ?it/s]

Downloading:   0%|          | 0.00/1.08M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/803k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.60M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.38k [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/transformers/models/marian/tokenization_marian.py:194: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


Downloading:   0%|          | 0.00/307M [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/transformers/generation_utils.py:1364: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 512 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  UserWarning,


  0%|          | 0/4100 [00:00<?, ?it/s]

In [12]:
df.sample(10)

,Unnamed: 0,en,ru,opus-mt,googletrans,opus-mt_bz=5,opus-mt_bz=10
1651,1651,Pam Warhurst: How we can eat our landscapes,Пэм Ворхёрст: Как сделать ландшафт съедобным,Pam Worhurst: How to make landscape edible,Pam Voorhurst: How to make a landscape edible,Pam Worhurst: How to make landscape edible,Pam Worhurst: How to make landscape edible
808,808,"You can't understand climate change in pieces,...","«Нельзя понять изменение климата по частям», —...","""You can't understand climate change in parts,...","""You can't understand climate change in pieces...","""You can't understand climate change in parts,...","""You can't understand climate change in parts,..."
2974,2974,"Ryan Lobo has traveled the world, taking photo...","Райан Лобо путешествовал по миру, делая фотогр...",Ryan Lobo traveled around the world taking pic...,Ryan Lobo has traveled the world taking photog...,Ryan Lobo traveled around the world taking pic...,Ryan Lobo traveled around the world taking pic...
1328,1328,What motivates us to work? Contrary to convent...,Что мотивирует на работу? Вопреки общепринятом...,What motivates you to work? Contrary to genera...,What motivates you to work? Contrary to popula...,What motivates you to work? Contrary to conven...,What motivates you to work? Contrary to conven...
1873,1873,Marco Tempest: A magical tale (with augmented ...,Марко Темпест: Волшебная сказка с дополненной ...,Marco Tempest: A magical fairy tale with an ad...,Marco Tempest: Fairy Tale with Augmented Reality,Marco Tempest: A magical fairy tale with an ad...,Marco Tempest: A magical fairy tale with an ad...
3522,3522,Psychologist Jonathan Haidt studies the five m...,Психолог Джонатан Хайдт исследует 5 моральных ...,Psychologist Jonathan Hydt explores five moral...,Psychologist Jonathan Haidt explores 5 moral v...,Psychologist Jonathan Hydt explores five moral...,Psychologist Jonathan Hydt explores five moral...
2820,2820,Drugs alone can't stop disease in sub-Saharan ...,Лекарства сами по себе не могут остановить бол...,Medicines alone cannot stop diseases in sub-Sa...,Drugs alone cannot stop disease in sub-Saharan...,Medicines alone cannot stop diseases in sub-Sa...,Medicines alone cannot stop diseases in sub-Sa...
2389,2389,"Isabel Behncke: Evolution's gift of play, from...",Изабель Бенке: эволюционный дар игры: от карли...,Isabel Benke: The evolutionary gift of the gam...,Isabelle Behnke: The Evolutionary Gift of Play...,Isabel Benke: The evolutionary gift of the gam...,Isabel Benke: The evolutionary gift of the gam...
2939,2939,Ravin Agrawal: 10 young Indian artists to watch,"Равин Агравал: 10 молодых художников Индии, ко...",Ravin Agraval: 10 young artists in India worth...,Ravin Agrawal: 10 young Indian artists worth s...,Ravin Agraval: 10 young artists in India worth...,Ravin Agraval: 10 young artists in India worth...
2354,2354,"Local politics -- schools, zoning, council ele...","Местная политика -- школы, зонирование, выборы...","Local politics -- schools, zoning, council ele...","Local politics -- schools, zoning, council ele...","Local politics -- schools, zoning, council ele...","Local politics -- schools, zoning, council ele..."


# 2. **[3 балла]** Оценка качества сгенерированных текстов

В рамках этого задания вам необходимо построить набор классификаторов, помогающий фильтровать выходы моделей машинного перевода тем самым улучшая качество. Задание состоит из нескольких этапов. 

### **[0.5 балла]** Выберите готовую модель машинного перевода из библиотеки EasyNMT, или любую другую, способную переводить тексты с английского языка на русский. Загрузите датасет [tatoeba](https://huggingface.co/datasets/tatoeba).

### С помощью готовой модели переведите тексты с английского на русский. Сделайте так, чтобы для одного примера на английском языке модель порождала несколько гипотез перевода на русский язык (например, можно взять различные значения параметра beam_size). Если инференс занимает много времени - возьмите первые 100 тысяч строчек датасета.

In [ ]:
# ! pip install pandas datasets

raw_data = load_dataset("tatoeba", lang1="en", lang2="ru")

df = pd.DataFrame(raw_data['train']['translation'])
print(f"Total samples: {len(df)}")
print(f"English unique samples: {df.en.nunique()}")

df.head(3)

Total samples: 523656
English unique samples: 370813


,en,ru
0,For once in my life I'm doing a good deed... A...,Один раз в жизни я делаю хорошее дело... И оно...
1,Let's try something.,Давайте что-нибудь попробуем!
2,Let's try something.,Давайте что-нибудь попробуем.


In [ ]:
# < your code here >

### **[0.5 балла]** Оцените качество перевода используя одну из известных вам метрик. Обратите внимание - примеры на английском языке **могут содержать несколько вариантов перевода** на русский. Учитывайте это при подсчете метрик.

In [ ]:
# < your code here >

### **[1 балл]** Используя предобученную модель [ruRoBERTa-large-rucola](https://huggingface.co/RussianNLP/ruRoBERTa-large-rucola) отранжируйте гипотезы машинного перевода для каждого из примеров на английском языке. Эта модель предсказывает вероятность того, что входное предложение приемлемо (грамматически верно) в русском языке. Вместо модели ruRoBERTa-large-rucola можно обучить свою на датасетах [RuATD](https://github.com/dialogue-evaluation/RuATD) и/или [RuCoLA](https://rucola-benchmark.com/) **[+1 балл (бонус)]**.

In [ ]:
# < your code here >

### **[1 балл]** Оцените качество перевода с учетом ранжирования гипотез, сравните с тем, что было раньше. Насколько сильно улучшилось/ухудшилось качество? Почему так произошло? Сделайте выводы о проделанном эксперименте. Продемонстрируйте примеры, на которых модель работает хорошо и на которых работает плохо (как с точки зрения метрик, так и с точки зрения вашего чувства прекрасного).

In [ ]:
# < your code here >

< your text here >

# 3. **[4 балла]** Обучение собственной модели машинного перевода

В этом задании вам предлагается самостоятельно обучить модель машинного перевода на основе одной из предобученных transformer-based моделей. Будем обучать модель, способную переводить текст с английского языка на турецкий. Для этого возьмем уже знакомый датасет [tatoeba](https://huggingface.co/datasets/tatoeba), разобьем данные на трейн, валидацию и тест (менять это разбиение нельзя!).

Полезные ссылки: [раз](https://huggingface.co/docs/transformers/tasks/translation), [два](https://towardsdatascience.com/how-to-train-an-mt5-model-for-translation-with-simple-transformers-30ba5fa66c5f), [три](https://towardsdatascience.com/data-augmentation-in-nlp-using-back-translation-with-marianmt-a8939dfea50a), [четыре](https://jalammar.github.io/visualizing-neural-machine-translation-mechanics-of-seq2seq-models-with-attention/), [пять](https://towardsdatascience.com/day-1-2-attention-seq2seq-models-65df3f49e263), [шесть](https://nlp.seas.harvard.edu/2018/04/03/attention.html)

In [ ]:
# ! pip install datasets pandas

from datasets import load_dataset
import pandas as pd

raw_data = load_dataset("tatoeba", lang1="en", lang2="tr")

df = pd.DataFrame(raw_data['train']['translation'])
print(f"Dataset contains {len(df)} samples")
df.head(3)

Dataset contains 672230 samples


,en,tr
0,Let's try something.,Bir şeyler deneyelim!
1,I have to go to sleep.,Yatmaya gitmek zorundayım.
2,I have to go to sleep.,Uyumam lazım.


In [ ]:
 # ! pip install scikit-learn
 
from sklearn.model_selection import train_test_split as tts

unique_queries = sorted(df.en.unique().tolist())
 
queries_train_val, queries_test = tts(unique_queries, test_size=0.2, random_state=41)
queries_train, queries_val = tts(queries_train_val, test_size=0.2, random_state=42)

assert set(queries_train) & set(queries_val) == set()
assert set(queries_train) & set(queries_test) == set()
assert set(queries_test) & set(queries_val) == set()

len(queries_train), len(queries_test), len(queries_val)

(409898, 128094, 102475)

In [ ]:
train_df = df.query("en in @queries_train").reset_index(drop=True)
val_df = df.query("en in @queries_val").reset_index(drop=True)
test_df = df.query("en in @queries_test").reset_index(drop=True)

### **[2 балла]** Обучите NMT модель используя обучающие, валидационные и тестовые данные **строго по назначению**. Можно воспользоваться готовыми фреймворками ([huggingface](https://huggingface.co/), [simpletransformers](https://simpletransformers.ai/)) или пайплайнами (например, [этим](https://github.com/devjwsong/transformer-translator-pytorch)). 


In [ ]:
# < your code here >

### **[1 балл]** Подберите подходящую метрику качества перевода. Постройте график функции потерь и график качества на обучении и валидации. Оцените обученную модель на тестовых данных с помощью выбранной метрики. Сделайте выводы о качестве модели исходя из полученных метрик.



In [ ]:
# < your code here >

< your text here >

### **[1 балл]** На основе нескольких пар из тестовой выборки визуализируйте работу механизма attention на различных слоях при помощи heatmap. Опишите полученную визуализацию, на какие слова модель обращает бо'льшее "внимание" при переводе?


In [ ]:
# < your code here >

< your text here >

### **[+2 балла (бонус)]** Попробуйте улучшить качество получившейся модели. Несколько идей для улучшений: взять более продвинутую модель, добавить больше данных в обучение (с помощью аугментации исходных данных или поиска новых), запустить обучение с другими гиперпараметрами, предобработать обучающие данные и др. Проведите аналогичный пункту 1 эксперимент, оцените качество модели. Продемонстрируйте примеры, из тестового сета на которых новая модель работает лучше старой. 

In [ ]:
# < your code here >